# Custom Indicators and Signal Filtering
Learn how to create proprietary technical indicators, apply signal filtering techniques, and leverage machine learning to enhance indicator power.

## Topics
- Creating proprietary indicators
- Signal filtering
- Machine learning enhanced indicators

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## 1. Designing a Customized Indicator
We'll design a volume-weighted momentum indicator as an example.

In [ ]:
def volume_momentum_indicator(price, volume, window=14):
    """Compute volume-weighted momentum: (price change * volume) rolling sum / rolling sum of volume."""
    returns = price.diff()
    weighted = returns * volume
    vw_momentum = weighted.rolling(window).sum() / volume.rolling(window).sum()
    return vw_momentum

In [ ]:
# Sample data and plot custom indicator
import yfinance as yf
data = yf.download('AAPL', period='6mo', interval='1d')
data['VW_Momentum'] = volume_momentum_indicator(data['Close'], data['Volume'])
plt.figure(figsize=(12,4))
plt.plot(data['VW_Momentum'], label='VW Momentum')
plt.legend()
plt.title('Volume-Weighted Momentum Indicator')
plt.show()

## 2. Signal Filtering Techniques
Smooth and refine generated signals to reduce noise and false triggers.

In [ ]:
# Apply exponential moving average filter
data['Signal'] = np.where(data['VW_Momentum'] > data['VW_Momentum'].ewm(span=14).mean(), 1, 0)
# Plot filtered signal
plt.figure(figsize=(12,4))
plt.plot(data['VW_Momentum'], label='VW Momentum', alpha=0.5)
plt.plot(data['VW_Momentum'].ewm(span=14).mean(), label='EMA Filter', linestyle='--')
plt.legend()
plt.show()

## 3. Machine Learning Enhanced Indicators
Use ML classifiers to generate signals based on multiple indicator features.

In [ ]:
# Feature engineering
delta = data['Close'].diff()

Compute RSI

up = delta.clip(lower=0)
down = -delta.clip(upper=0)
gain = up.rolling(14).mean()
loss = down.rolling(14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))
features = ['VW_Momentum', 'RSI']
data = data.dropna()
X = data[features]
y = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, preds))

## 4. Backtesting the Strategy
Evaluate strategy performance using ML-enhanced signals.

In [ ]:
# Generate positions based on model predictions
backtest_data = data.iloc[X_train.shape[0]:].copy()  # align with X_test
backtest_data['ML_Signal'] = preds
# Calculate returns
backtest_data['Strategy_Return'] = backtest_data['ML_Signal'] * backtest_data['Close'].pct_change()
cumulative_return = (1 + backtest_data['Strategy_Return']).cumprod()
plt.figure(figsize=(12,4))
plt.plot(cumulative_return, label='ML Strategy')
plt.legend()
plt.title('Cumulative Returns of ML-Enhanced Strategy')
plt.show()